In [1]:
import os
import time
import math
import numpy as np
from IPython.utils import io
import pandas as pd
from tqdm import tqdm
from dataclasses import dataclass
from typing import List, Union 
import matplotlib.pyplot as plt
import matplotlib
from backtesting import Backtest, Strategy
import multiprocessing as mp
from backtesting.lib import crossover
import backtesting
import sambo
import joblib
import ta
import torch
import torch.nn as nn
from PERMUTATIONS import get_permutation
from strategy import ARSIstrat
import MLP_WFSIG_functions as MLP

/Users/matteogiori/trycloneinsider/TradingBot/.venv313/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                        Open    High     Low    Close   Volume
time                                                          
2018-02-01 14:30:00  168.120  169.30  168.10  168.860   778080
2018-02-01 15:30:00  168.890  169.56  168.77  169.000   369077
2018-02-01 16:30:00  169.030  169.56  169.01  169.350   308644
2018-02-01 17:30:00  169.350  169.72  169.33  169.610   233919
2018-02-01 18:30:00  169.610  169.62  168.15  168.500   536976
2018-02-01 19:30:00  168.530  168.84  168.16  168.180   472291
2018-02-01 20:30:00  168.180  168.28  167.43  167.970   881835

In [2]:
csv_path = 'BATS_QQQ, 60_a45be.csv'

full_df = MLP.load_data(csv_path, start='2018-01-01', end_excl='2024-01-01')
df_train = full_df.loc['2018-01-01':'2021-12-31']
df_test = full_df.loc['2023-01-01':'2023-12-31']

In [3]:
def WF_w_Backtesting(df : pd.DataFrame, nbr_perm : int = 200, train_wind : int = 365 * 7) :
    mask2 = ((df.index.year >= 2016) & (df.index.year <  2022))
    train_df = df.loc[mask2]
    train_df['r'] = np.log(train_df['Close']).diff().shift(-1)
    train_window = train_wind 
    train_df['MA_wf_signal'] = MLP.walkforward_ARSI(train_df, train_lookback=train_window)

    MA_rets = train_df['MA_wf_signal'] * train_df['r']
    real_wf_pf = MA_rets[MA_rets > 0].sum() / MA_rets[MA_rets < 0].abs().sum()

    n_permutations = nbr_perm
    perm_better_count = 1
    permuted_sr = []
    print("Walkforward MCPT")
    for perm_i in tqdm(range(1, n_permutations)):
        wf_perm = get_permutation(train_df, start_index=train_window)
        
        wf_perm['r'] = np.log(wf_perm['Close']).diff().shift(-1) 
        wf_perm_sig = MLP.walkforward_ARSI(wf_perm, train_lookback=train_window)
        perm_rets = wf_perm['r'] * wf_perm_sig
        perm_pf = perm_rets[perm_rets > 0].sum() / perm_rets[perm_rets < 0].abs().sum()
        
        if perm_pf >= real_wf_pf:
            perm_better_count += 1

        permuted_sr.append(perm_pf)


    walkforward_mcpt_pval = perm_better_count / n_permutations
    print(f"Walkforward MCPT P-Value: {walkforward_mcpt_pval}")
    plt.style.use('dark_background')
    pd.Series(permuted_sr).hist(color='blue', label='Permutations')
    plt.axvline(real_wf_pf, color='red', label='Real')
    plt.xlabel("Profit Factor")
    plt.title(f"Walkforward MCPT. P-Value: {walkforward_mcpt_pval}")
    plt.grid(False)
    plt.legend()
    plt.show()

In [ ]:
WF_w_Backtesting(full_df,50)

/var/folders/6x/40f088zn6lb1tw1zn99534440000gn/T/ipykernel_16935/439652531.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['r'] = np.log(train_df['Close']).diff().shift(-1)
/var/folders/6x/40f088zn6lb1tw1zn99534440000gn/T/ipykernel_16935/439652531.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['MA_wf_signal'] = MLP.walkforward_ARSI(train_df, train_lookback=train_window)


Walkforward MCPT


 69%|██████▉   | 34/49 [8:27:43<3:09:22, 757.52s/it]  